In [658]:
import pandas as pd
import datetime as dt
import numpy as np
from tqdm import tqdm

# Предобработка данных

In [582]:
# Чтение данных

df = pd.read_csv("transactions.csv")

In [583]:
# Переименование названий колонок

df.rename({"Дата продажи":"date", 
           "Чек":"rec_num", 
           "Клиент":"client", 
           "Товар":"item", 
           "Товарная группа":"item_group", 
           "Кол-во":"amount", 
           "Сумма":"price"}, 
           axis=1,
           inplace=True)

In [584]:
# Обработка даты. Првиедение ее к виду "01-%m-%Y"

df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y")
df["year_month"] = df["date"].dt.strftime('%m-%Y')
df["year_month"] = pd.to_datetime(df["year_month"], format="%m-%Y")

# Скользящее окно

In [298]:
# Группировка

grouped_df = df.groupby(["year_month", "item_group"], as_index=False).agg("sum")

In [299]:
# Сортировка

grouped_df.sort_values(by=["item_group", "year_month"], inplace=True)

In [300]:
# Реализация скользящего окна

grouped_df["amount_1"] = grouped_df.groupby(["item_group"], as_index=False)["amount"].shift(1)
grouped_df["amount_2"] = grouped_df.groupby(["item_group"], as_index=False)["amount"].shift(2)
grouped_df["amount_3"] = grouped_df.groupby(["item_group"], as_index=False)["amount"].shift(3)

In [301]:
# Фильтрация от строк, где нет данных

grouped_df = grouped_df[grouped_df["year_month"] >= "2017-12-01"]

In [302]:
grouped_df

,year_month,item_group,amount,price,amount_1,amount_2,amount_3
51,2017-12-01,Ванная комната,399,986907,510.0,413.0,326.0
68,2018-01-01,Ванная комната,536,1412932,399.0,510.0,413.0
85,2018-02-01,Ванная комната,624,1474027,536.0,399.0,510.0
102,2018-03-01,Ванная комната,526,1094837,624.0,536.0,399.0
119,2018-04-01,Ванная комната,494,1219927,526.0,624.0,536.0
...,...,...,...,...,...,...,...
373,2019-06-01,Электротовары,11895,1743978,12401.0,10681.0,11699.0
390,2019-07-01,Электротовары,13078,1756495,11895.0,12401.0,10681.0
407,2019-08-01,Электротовары,13500,1868637,13078.0,11895.0,12401.0
424,2019-09-01,Электротовары,14643,1892387,13500.0,13078.0,11895.0


In [303]:
# Расчет прогноза

grouped_df["forecast"] = round((grouped_df.amount_1 + grouped_df.amount_2 + grouped_df.amount_3)/3, 2)

In [304]:
# Расчет MAPE 

def MAPE_count(df):
    """
    Принимает таблицу, и считает MAPE
    """
    return round(sum(abs(df.forecast - df.amount) / df.amount)/len(df) * 100, 2)

res_df = pd.DataFrame({"MAPE_%":grouped_df.groupby("item_group").apply(MAPE_count)})   

In [305]:
"""
Ответ на вопрос: Какое значение MAPE имеет товарная группа Освещение? Ответокруглите до двух знаков после запятой.
33.79%
"""
res_df

,MAPE_%
item_group,
Ванная комната,18.37
Древесно-плитные материалы,21.55
Инструменты,16.53
Кафель и плитка,20.55
Комнатные растения и цветы,77.78
Лаки и краски,16.15
Обои,18.18
Оборудование для сада и дачи,46.91
Окна и двери,20.86


In [306]:
# Реальный прогноз на ноябрь 2019
"""
Ответ на вопрос: Сколько, согласно модели прогноза по среднему, будет продано товаров группы с самой минимальной 
ошибкой MAPE в ноябре 2019 года?

12322 - столярные изделия
"""
grouped_df[grouped_df["year_month"] >= "2019-08-01"].groupby("item_group")["amount"].agg("mean")

item_group
Ванная комната                   1107.666667
Древесно-плитные материалы       2104.333333
Инструменты                      4839.333333
Кафель и плитка                  4502.666667
Комнатные растения и цветы       5118.333333
Лаки и краски                   14187.000000
Обои                             1560.666667
Оборудование для сада и дачи     8495.000000
Окна и двери                     2635.333333
Освещение                        2270.000000
Садовая мебель                    698.333333
Сантехника                       7783.000000
Скобяные изделия                12750.333333
Стойматериалы                   31002.666667
Столярные изделия               12322.000000
Товары для дачи и отдыха          870.333333
Электротовары                   14300.000000
Name: amount, dtype: float64

# Квантование

In [459]:
df

,date,rec_num,client,item,item_group,amount,price,year_month
0,2017-09-01,code000000001,client13166,sku8444,Скобяные изделия,1,29,2017-09-01
1,2017-09-01,code000000001,client13166,sku12545,Оборудование для сада и дачи,1,329,2017-09-01
2,2017-09-01,code000000001,client13166,sku3391,Инструменты,1,169,2017-09-01
3,2017-09-01,code000000001,client13166,sku20444,Инструменты,2,578,2017-09-01
4,2017-09-01,code000000002,client1239,sku29959,Скобяные изделия,1,329,2017-09-01
...,...,...,...,...,...,...,...,...
1008683,2019-10-31,code000290228,client25258,sku5837,Скобяные изделия,1,89,2019-10-01
1008684,2019-10-31,code000290228,client25258,sku26161,Стойматериалы,2,598,2019-10-01
1008685,2019-10-31,code000290228,client25258,sku20658,Сантехника,1,159,2019-10-01
1008686,2019-10-31,code000290229,client22472,sku16069,Столярные изделия,1,85,2019-10-01


In [460]:
# Группировка по клиентам

grouped_df_with_clients = df.groupby(["client"], as_index=False)[["amount", "price", "date"]] \
.agg({"amount": "sum", "price":"sum", "date":"nunique"})

In [461]:
grouped_df_with_clients

,client,amount,price,date
0,client1,9,2273,1
1,client10,3,4757,1
2,client100,7,8063,3
3,client1000,309,44853,16
4,client10000,5,8495,1
...,...,...,...,...
42741,client9995,3,3677,2
42742,client9996,1,2499,1
42743,client9997,24,17691,3
42744,client9998,248,38666,32


In [462]:
# Квантование. Доли клиентов в группах в процентах и абсолютных значениях

grouped_df_with_clients["bins_equal"] = pd.cut(grouped_df_with_clients.date, bins=[0, 1, 2, 4, 9, np.inf], right=True)
q_df = pd.DataFrame({"clients": grouped_df_with_clients.sort_values(by="date").bins_equal.value_counts()})
q_df["percent"] = round(q_df.clients/len(grouped_df_with_clients) * 100, 0)
q_df = q_df.sort_index()
q_df

,clients,percent
"(0.0, 1.0]",12859,30.0
"(1.0, 2.0]",6451,15.0
"(2.0, 4.0]",7286,17.0
"(4.0, 9.0]",8250,19.0
"(9.0, inf]",7900,18.0


In [467]:
# Квантование по количеству отрезков(15)
"""
Вопрос: Ответьте на вопрос: Сколько клиентов (кол-во человек) попало в последний
интервал при разбиении методом "количество".
Ответ - 1
"""

grouped_df_with_clients["bins_15"] = pd.cut(grouped_df_with_clients.date, bins=15, right=True)
grouped_df_with_clients.sort_values(by="date").bins_15.value_counts() 

(0.827, 12.533]       37193
(12.533, 24.067]       3800
(24.067, 35.6]         1026
(35.6, 47.133]          418
(47.133, 58.667]        134
(58.667, 70.2]           72
(70.2, 81.733]           47
(81.733, 93.267]         24
(93.267, 104.8]          10
(104.8, 116.333]          9
(116.333, 127.867]        4
(150.933, 162.467]        3
(162.467, 174.0]          3
(139.4, 150.933]          2
(127.867, 139.4]          1
Name: bins_15, dtype: int64

# Задача любимая группа
Задача: Рассчитайте для каждого клиента его две «любимых» товарных группы. «Любимая» группа – та, на которую
сделаны покупки на максимальную сумму (за все визиты).

Ответьте на вопросы:

- У какого числа клиентов Любимая группа 1 это "Освещение"? Имейте ввиду, что может возникнуть ситуация, когда у клиента
совпадают суммы по нескольким группам. Предусмотрите это, отсортировав группы по алфавиту.

- У какого числа клиентов нет любимой группы 2?

- Если вы правильно спроектировали подмодель для расчета любимых групп, вы быстро сможете получить ответ на следующий вопрос: У какого числа клиентов Любимая группа 1 это "Освещение", если расчет производить не по сумме, а количеству товара?

In [681]:
# Группировка по клиентам и товарным группам

df_grouped_by_item_group = df.groupby(["client", "item_group"], as_index=False)[["amount", "price"]].sum()
df_grouped_by_item_group

,client,item_group,amount,price
0,client1,Лаки и краски,4,738
1,client1,Стойматериалы,5,1535
2,client10,Инструменты,3,4757
3,client100,Лаки и краски,4,266
4,client100,Оборудование для сада и дачи,1,7299
...,...,...,...,...
205366,client9999,Комнатные растения и цветы,13,311
205367,client9999,Лаки и краски,1,182
205368,client9999,Оборудование для сада и дачи,5,515
205369,client9999,Сантехника,5,4695


In [682]:
# # Сортировка по клиентам и стоимости товаров

# df_grouped_by_item_group.sort_values(by=["client", "price"], ascending=False, inplace=True)
# df_grouped_by_item_group

# df_grouped_by_item_group.sort_values(by=["client", "amount"], ascending=False, inplace=True)
# df_grouped_by_item_group

In [683]:
# Сортировка по товарным группам

df_grouped_by_item_group.sort_values(by=["client", "item_group"])

,client,item_group,amount,price
0,client1,Лаки и краски,4,738
1,client1,Стойматериалы,5,1535
2,client10,Инструменты,3,4757
3,client100,Лаки и краски,4,266
4,client100,Оборудование для сада и дачи,1,7299
...,...,...,...,...
205366,client9999,Комнатные растения и цветы,13,311
205367,client9999,Лаки и краски,1,182
205368,client9999,Оборудование для сада и дачи,5,515
205369,client9999,Сантехника,5,4695


In [684]:
# Присваивание степени того, насколько товар любимый по цене и по количеству

df_grouped_by_item_group["item_group_rank"] = df_grouped_by_item_group.groupby("client")["price"] \
.rank(method="first", ascending=False)
df_grouped_by_item_group.head(10)

# df_grouped_by_item_group["item_group_rank"] = df_grouped_by_item_group.groupby("client")["amount"] \
# .rank(method="first", ascending=False)
# df_grouped_by_item_group.head(50)



,client,item_group,amount,price,item_group_rank
0,client1,Лаки и краски,4,738,2.0
1,client1,Стойматериалы,5,1535,1.0
2,client10,Инструменты,3,4757,1.0
3,client100,Лаки и краски,4,266,3.0
4,client100,Оборудование для сада и дачи,1,7299,1.0
5,client100,Освещение,1,299,2.0
6,client100,Электротовары,1,199,4.0
7,client1000,Инструменты,2,84,5.0
8,client1000,Сантехника,4,7896,2.0
9,client1000,Скобяные изделия,89,6483,3.0


In [685]:
# Фильтрация не нужных строк

df_grouped_by_item_group = df_grouped_by_item_group[df_grouped_by_item_group["item_group_rank"] <= 2.0]


In [686]:
# Создание результирующей таблице по любимым товарамным категориям (по цене)

f_groups_by_spending = pd.DataFrame(columns=["f_group_1", "f_group_2"])

for index, row in tqdm(df_grouped_by_item_group.iterrows()):
    item_group = row.item_group
    
    if row.item_group_rank == 1.0:
        f_groups_by_spending.loc[row.client, "f_group_1"] = item_group
        
    if row.item_group_rank == 2.0:
        f_groups_by_spending.loc[row.client, "f_group_2"] = item_group
    

78906it [00:43, 1811.92it/s]


In [688]:
f_groups_by_spending.f_group_1.value_counts(dropna=False)

Оборудование для сада и дачи    5270
Инструменты                     5223
Стойматериалы                   4321
Лаки и краски                   3389
Ванная комната                  3145
Столярные изделия               2511
Освещение                       2366
Обои                            2203
Комнатные растения и цветы      1993
Электротовары                   1984
Сантехника                      1910
Кафель и плитка                 1800
Садовая мебель                  1662
Товары для дачи и отдыха        1474
Скобяные изделия                1316
Окна и двери                    1109
Древесно-плитные материалы      1070
Name: f_group_1, dtype: int64

In [689]:
f_groups_by_spending.f_group_2.value_counts(dropna=False)

NaN                             6586
Лаки и краски                   4846
Оборудование для сада и дачи    3988
Электротовары                   3436
Стойматериалы                   3414
Скобяные изделия                2588
Инструменты                     2461
Комнатные растения и цветы      2285
Сантехника                      2084
Освещение                       2037
Ванная комната                  1782
Столярные изделия               1596
Товары для дачи и отдыха        1169
Обои                            1054
Кафель и плитка                  965
Древесно-плитные материалы       956
Садовая мебель                   812
Окна и двери                     687
Name: f_group_2, dtype: int64